<a href="https://colab.research.google.com/github/ariflaksito/naive-bayes/blob/master/02_nb_studentpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes
Prediksi mahasiswa lulus tepat waktu/tidak

## Proses Cleaning Data

In [0]:
# import library
import pandas as pd
import numpy as np

In [0]:
# Membaca data dari file excel
data = pd.read_excel('Data-5K-shared.xlsx')

In [3]:
data.head()

,ANGKATAN,JK,TGLAHIR,TGLULUS,IPK1,IPK2,IPK3,IPK4,SKS
0,2013,L,1993-10-14 00:00:00,2016-09-26,3.500000,3.833333,4.000000,3.500000,144
1,2013,L,1995-05-06 00:00:00,2017-03-25,3.500000,3.750000,3.333333,3.250000,144
2,2013,L,1995-01-31 00:00:00,2016-09-26,3.166667,3.083333,3.333333,3.250000,144
3,2013,P,1996-01-22 00:00:00,2016-12-24,3.500000,3.250000,3.500000,3.416667,144
4,2013,L,1995-06-15 00:00:00,NaT,3.083333,3.083333,3.000000,2.000000,126


In [4]:
data.shape

(5125, 9)

In [5]:
data.dtypes

ANGKATAN             int64
JK                  object
TGLAHIR             object
TGLULUS     datetime64[ns]
IPK1               float64
IPK2               float64
IPK3               float64
IPK4               float64
SKS                  int64
dtype: object

In [6]:
# Melihat data yang tglulus kosong
# Diasumsikan sampai saat ini tidak lulus
data[data.TGLULUS.isnull()]

,ANGKATAN,JK,TGLAHIR,TGLULUS,IPK1,IPK2,IPK3,IPK4,SKS
4,2013,L,1995-06-15 00:00:00,NaT,3.083333,3.083333,3.000000,2.000000,126
7,2013,L,2013-02-23 00:00:00,NaT,3.333333,3.416667,3.750000,3.333333,146
8,2013,L,1993-09-24 00:00:00,NaT,3.333333,3.250000,3.416667,3.166667,140
11,2013,L,1994-11-15 00:00:00,NaT,3.083333,3.083333,3.166667,2.833333,138
12,2013,L,1993-10-27 00:00:00,NaT,3.166667,3.000000,3.416667,0.000000,72
...,...,...,...,...,...,...,...,...,...
5109,2005,L,1985-07-06 00:00:00,NaT,2.833333,3.428571,2.416667,0.000000,140
5113,2005,L,1987-12-17 00:00:00,NaT,0.416667,1.571429,0.000000,0.000000,38
5114,2005,P,1987-09-03 00:00:00,NaT,3.750000,1.642857,0.000000,0.000000,42
5115,2005,P,1987-05-10 00:00:00,NaT,3.333333,3.142857,2.333333,0.000000,138


In [0]:
# Gunakan data dengan tglulus tidak kosong
clean_data = data[data.TGLULUS.notnull()]

In [8]:
clean_data.shape

(3667, 9)

In [9]:
# Cek juga data angkatan yang kosong
clean_data[clean_data.ANGKATAN.isnull()]

,ANGKATAN,JK,TGLAHIR,TGLULUS,IPK1,IPK2,IPK3,IPK4,SKS


In [0]:
# Gunakan data dengan angkatan tidak kosong
fix_data = clean_data[clean_data.ANGKATAN.notnull()]

In [0]:
# Tambah atribute Tglmulai diambil dari angkatan mhs, diasumsikan 1 agustus
fix_data["TGLMULAI"] = pd.to_datetime(dict(year=fix_data.ANGKATAN, month=8, day=1))

In [12]:
fix_data.head()

,ANGKATAN,JK,TGLAHIR,TGLULUS,IPK1,IPK2,IPK3,IPK4,SKS,TGLMULAI
0,2013,L,1993-10-14 00:00:00,2016-09-26,3.500000,3.833333,4.000000,3.500000,144,2013-08-01
1,2013,L,1995-05-06 00:00:00,2017-03-25,3.500000,3.750000,3.333333,3.250000,144,2013-08-01
2,2013,L,1995-01-31 00:00:00,2016-09-26,3.166667,3.083333,3.333333,3.250000,144,2013-08-01
3,2013,P,1996-01-22 00:00:00,2016-12-24,3.500000,3.250000,3.500000,3.416667,144,2013-08-01
5,2013,L,1994-12-11 00:00:00,2017-03-25,3.250000,3.166667,3.250000,3.166667,144,2013-08-01


In [0]:
# Hitung masa studi(bulan)
fix_data['MSTUDI'] = (fix_data['TGLULUS']-fix_data['TGLMULAI'])/np.timedelta64(1, 'M')

In [14]:
fix_data.head(3668)

,ANGKATAN,JK,TGLAHIR,TGLULUS,IPK1,IPK2,IPK3,IPK4,SKS,TGLMULAI,MSTUDI
0,2013,L,1993-10-14 00:00:00,2016-09-26,3.500000,3.833333,4.000000,3.500000,144,2013-08-01,37.848826
1,2013,L,1995-05-06 00:00:00,2017-03-25,3.500000,3.750000,3.333333,3.250000,144,2013-08-01,43.762706
2,2013,L,1995-01-31 00:00:00,2016-09-26,3.166667,3.083333,3.333333,3.250000,144,2013-08-01,37.848826
3,2013,P,1996-01-22 00:00:00,2016-12-24,3.500000,3.250000,3.500000,3.416667,144,2013-08-01,40.772911
5,2013,L,1994-12-11 00:00:00,2017-03-25,3.250000,3.166667,3.250000,3.166667,144,2013-08-01,43.762706
...,...,...,...,...,...,...,...,...,...,...,...
5119,2005,P,1986-12-16 00:00:00,2009-03-25,3.666667,3.357143,3.384615,0.000000,146,2005-08-01,43.762706
5120,2005,P,1987-03-03 00:00:00,2009-09-12,3.083333,3.428571,2.923077,0.000000,144,2005-08-01,49.380891
5121,2005,L,1987-02-20 00:00:00,2013-03-25,2.333333,2.500000,1.454545,0.000000,144,2005-08-01,91.763691
5122,2005,L,1986-02-13 00:00:00,2011-03-25,2.500000,2.642857,2.153846,0.000000,144,2005-08-01,67.746771


In [0]:
# Tambat atribut status dimana <= 48 bulan = lulus tepat waktu
fix_data['STATUS'] = np.where(fix_data['MSTUDI']>48, 'TDK TEPAT', 'TEPAT')

In [16]:
fix_data.head(3668)

,ANGKATAN,JK,TGLAHIR,TGLULUS,IPK1,IPK2,IPK3,IPK4,SKS,TGLMULAI,MSTUDI,STATUS
0,2013,L,1993-10-14 00:00:00,2016-09-26,3.500000,3.833333,4.000000,3.500000,144,2013-08-01,37.848826,TEPAT
1,2013,L,1995-05-06 00:00:00,2017-03-25,3.500000,3.750000,3.333333,3.250000,144,2013-08-01,43.762706,TEPAT
2,2013,L,1995-01-31 00:00:00,2016-09-26,3.166667,3.083333,3.333333,3.250000,144,2013-08-01,37.848826,TEPAT
3,2013,P,1996-01-22 00:00:00,2016-12-24,3.500000,3.250000,3.500000,3.416667,144,2013-08-01,40.772911,TEPAT
5,2013,L,1994-12-11 00:00:00,2017-03-25,3.250000,3.166667,3.250000,3.166667,144,2013-08-01,43.762706,TEPAT
...,...,...,...,...,...,...,...,...,...,...,...,...
5119,2005,P,1986-12-16 00:00:00,2009-03-25,3.666667,3.357143,3.384615,0.000000,146,2005-08-01,43.762706,TEPAT
5120,2005,P,1987-03-03 00:00:00,2009-09-12,3.083333,3.428571,2.923077,0.000000,144,2005-08-01,49.380891,TDK TEPAT
5121,2005,L,1987-02-20 00:00:00,2013-03-25,2.333333,2.500000,1.454545,0.000000,144,2005-08-01,91.763691,TDK TEPAT
5122,2005,L,1986-02-13 00:00:00,2011-03-25,2.500000,2.642857,2.153846,0.000000,144,2005-08-01,67.746771,TDK TEPAT


In [0]:
# Export data ke Excel
fix_data.to_excel(r'FixData.xlsx',index = False)

## Transformasi data

In [0]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
fix_data['JK'] = number.fit_transform(fix_data['JK'])
fix_data['STATUS'] = number.fit_transform(fix_data['STATUS'])

In [0]:
# Set features dan target
features = ["JK", "IPK1", "IPK2", "IPK3", "IPK4"]
target = "STATUS"

In [0]:
# Set training data dan testing data: 70% - 30%
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(fix_data[features],
    fix_data[target],
    test_size = 0.30,
    random_state = 54)

## Model Naive Bayes

In [20]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(features_train, target_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
pred = model.predict(features_test)

In [0]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(target_test, pred)

In [23]:
accuracy

0.633969118982743

In [24]:
confusion_matrix = pd.crosstab(target_test, pred, rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,286,310
1,93,412
